#Imports
Everyting with torch is for PyTorch which is our system for the neural network

Numpy, pandas, and matlab is for structuring and visualisation of data
%matplotlib and %config is used to show graphs in line in the google collab view

In [0]:
# imports related to PyTorch
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict

# python tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# other
import json
from PIL import Image

# google collab specific
from google.colab import drive

#Loading of data
We are using `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The dataset is split into three parts, training, validation, and testing. For the training, there has been applyed transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this we don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks we'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three set we'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225], calculated from the ImageNet images. These values will shift each color channel to be centered at 0 and range from -1 to 1.

#### Gets acces to the google drive folder
\* only run if using google collab

In [0]:
# gets acces to the google drive folder
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### getting the data
we are using the data loader class from PyTorch which means that the images should be arranged following this structure:

* class 'dog'
    * root/dog/xxx.png
    * root/dog/xxy.png
    * root/dog/xxz.png
* class 'cat'
    * root/cat/123.png
    * root/cat/nsdf3.png
    * root/cat/asd932_.png



In [0]:
# paths to data needs to get changed, for new data location 
path = '/content/gdrive/My Drive/cs3_Rasmussen/Collab/GreenMinds-recycling-bot/Fruits_data'

data_dir = {
    'train': path + '/train',
    'valid': path+ '/valid',
    'test': path + '/test'   
}

# Defines the transforms for the training, validation, and testing sets
data_transforms = {
    'train': transforms.Compose([
                transforms.RandomRotation(30),
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]),
    'valid': transforms.Compose([
                transforms.Resize(255),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]),
    'test': transforms.Compose([
                transforms.Resize(255),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
}

# Loads the datasets with ImageFolder
data_datasets = {
    'train': datasets.ImageFolder(data_dir['train'], transform=data_transforms['train']),
    'valid': datasets.ImageFolder(data_dir['valid'], transform=data_transforms['valid'])
 #   'test': datasets.ImageFolder(data_dir['test'], transform=data_transforms['test'])
}

# Using the image datasets and the trainforms, defines the dataloaders
# we are using batch_size of 32 to make sure not to run out of memory on google collab
data_dataloaders = {
    'train': torch.utils.data.DataLoader(data_datasets['train'], batch_size=32, shuffle=True),
    'valid': torch.utils.data.DataLoader(data_datasets['valid'], batch_size=32)
    #'test': torch.utils.data.DataLoader(data_datasets['test'], batch_size=32)
}

# Building the classifier

Getting our pretrained model

In [0]:
model = models.densenet121(pretrained=True)

Freezing the parameters so we wont backprop through them

In [0]:
for param in model.parameters():
    param.requires_gradu = False

Creating a new classifier for transfor learning

In [0]:
prediction_size = 6

classifier = nn.Sequential(OrderedDict([
    ('fc_1', nn.Linear(1024, 512)),
    ('relu_1', nn.ReLU()),
    ('dropout_1', nn.Dropout(.2)),
    ('fc_2', nn.Linear(512, 256)),
    ('relu_2', nn.ReLU()),
    ('dropout_2', nn.Dropout(.2)),
    ('fc_3', nn.Linear(256, prediction_size)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

Checks if cuda is enabled, and sends our model to the gpu if avaliable

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

RuntimeError: ignored

Defining hyper parameters

In [0]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

epochs = 1
steps = 0
print_every_step = 10

Creating our training loop

In [0]:
# sets our model to train mode
model.train()
# sends our model to the device (cpu/gpu)
model = model.to(device)

# for graphing
train_losses, test_losses = [], []

# runs through set amount of epochs
for epoch in range(epochs):
    # holds our runnning loss
    running_loss = 0

    # loops over our train data
    for inputs, labels in data_dataloaders['train']:
        # add 1 to the number of steps done
        steps += 1
        
        # moves input- and labels- tensors to the device (cpu/gpu)
        inputs, labels = inputs.to(device), labels.to(device)
        
        # resets our optimizer
        optimizer.zero_grad()
        # gets our log probabilities from our model
        logps = model.forward(inputs)
        # calculates the loss using our criterion
        loss = criterion(logps, labels)
        # backprobagates to update our weights
        loss.backward()
        optimizer.step()
        
        # adds our loss to the total loss, between validation
        running_loss += loss.item()
        
        # calculates our models current accuracy each x steps
        if steps % print_every_step == 0:
            # holds our test loss
            test_loss = 0
            # holds our total accuracy
            accuracy = 0
            
            # sets our model to eval, to stop dropout
            model.eval()
            
            # stops our optimizer for better performance
            with torch.no_grad():
                # loops through our validation data
                for inputs, labels in data_dataloaders['valid']:
                    # moves input- and labels- tensors to the device (cpu/gpu)
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    # gets our log probabilities from our model
                    logps = model.forward(inputs)
                    # calculates the loss using our criterion
                    batch_loss = criterion(logps, labels)
                    # adds our loss to the total loss validation loss
                    test_loss += batch_loss.item()
                    
                    # ---Calcualtes the accuracy
                    # converts to probabilities
                    ps = torch.exp(logps)
                    # gets the top prediction
                    top_p, top_class = ps.topk(1, dim=1)
                    # Check if it is equal to the label
                    equals = top_class == labels.view(*top_class.shape)
                    # adds our accuracy this step to the total validation accuracy
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            
            train_losses.append(running_loss/print_every_step)
            test_losses.append(test_loss/len(data_dataloaders['valid']))
            
            # Prints out data about our current model accuracy and losses
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every_step:.3f}.. "
                  f"Test loss: {test_loss/len(data_dataloaders['valid']):.3f}.. "
                  f"Test accuracy: {accuracy/len(data_dataloaders['valid']):.10f}")
            
            # resets our running loss
            running_loss = 0
            # resets our model to train mode
            model.train()

RuntimeError: ignored

#### Plots training vs validation loss
used to check for over / under fitting

In [0]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.ylim(bottom=0)
plt.legend(frameon=False)

# Validation on the test set

In [0]:
# holds our total test loss
test_loss = 0
# holds our total accuracy
accuracy = 0
# sends our model to the device (cpu/gpu)
model = model.to(device)
# sets our model to eval mode
model.eval()

# stops our optimizer for better performance
with torch.no_grad():
    # loops over our test data
    for inputs, labels in data_dataloaders['test']:
        # moves input- and labels- tensors to the device (cpu/gpu)
        inputs, labels = inputs.to(device), labels.to(device)
        
        # gets our log probabilities from our model
        logps = model.forward(inputs)
        
        # --- Calcualtes the accuracy
        # converts to probabilities
        ps = torch.exp(logps)
        # gets the top prediction
        top_p, top_class = ps.topk(1, dim = 1)
        # Check if it is equal to the label
        equals = top_class == labels.view(*top_class.shape)
        # adds our accuracy this step to the total test accuracy
        accuracy += torch.mean(equals.type(torch.FloatTensor))

# prints out the final result
print('Test accuracy: {}'.format(accuracy / len(data_dataloaders['test'])))